# Data preprocessing and Feature Engineering

In order to obtain a learnable dataset, features must be preprocessed and engineered in order to contain valuable learning data. This notebook will generate a PySpark RDD that contains the preprocessed dataset, ready for feature selection algorithms.

More than 20 features must be engineered, some ideas are:

Session-time related features:
* Weekday of the session
* Month of the session
* Season of the session
* Duration of the session
* Day period of the session

Session-item related features:
* Item with the most time spent on.
* Mean time per item.
* Variance time per item.
* Item revisit?

### Starting the Spark engine and loading the dataset.

In [1]:
import os 
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

# launch this cell if you have issues on windows with py4j (think about updating your PATH)
import sys
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

# starts a spark session from notebook

os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=4g  pyspark-shell"
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("load_explore") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/10 12:59:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# loads relevant datas in DataFrames
train_sessions = spark.read.load('../Data/train_sessions.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

train_purchases = spark.read.load('../Data/train_purchases.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

candidate_items = spark.read.load('../Data/candidate_items.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

item_features = spark.read.load('../Data/item_features.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

datas = [train_sessions, train_purchases, candidate_items, item_features]

## Group sessions and aggregate their duration.

Using a map reduce operation, we will start by computing the session duration in seconds. For that, we use a function that will convert the timestamp into seconds since the 1st January 1970 (UNIX time).

In [3]:
import datetime

def timestamp_to_unix(timestamp: str) -> int:
    '''Converts the timestamp, on a string format to the UNIX time
    '''
    try:
        date = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S.%f')
    except ValueError:
        date = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
    return int(date.timestamp())

posix_date_mapped_sessions = train_sessions.rdd.map(lambda x: (x.session_id, timestamp_to_unix(x.date))).cache()
sampled_sessions = posix_date_mapped_sessions.sample(False, 0.0001, seed=42)

In [4]:
sampled_sessions.take(1)

22/05/10 12:59:27 WARN BlockManager: Task 23 already completed, not releasing lock for rdd_45_0


[(4163, 1606579200)]

In [5]:
def reduce_min_max(x, y):
    if type(x) == tuple:
        return(max(x[0], y), min(x[1], y))
    return (max(x, y), min(x, y))

reduced_sessions = posix_date_mapped_sessions.reduceByKey(lambda x, y: reduce_min_max(x, y))
reduced_sessions.take(1)

[(24, (1582741472, 1582737768))]

In [6]:
def get_duration_seconds(tupl):
    if type(tupl[1]) is int:
        return (tupl[0], 0)
    else:
        return (tupl[0], tupl[1][0] - tupl[1][1])

time_sessions = reduced_sessions.map(lambda x: get_duration_seconds(x))

In [7]:
time_sessions.take(10)

[(24, 3704),
 (48, 657),
 (184, 0),
 (208, 0),
 (232, 0),
 (248, 1716),
 (352, 190),
 (376, 171),
 (384, 409),
 (464, 0)]

We cannot compute the sessions that only show one item, so we set the session duration to 0

## Group sessions and aggregate the number of unique AND total viewed items

We compute how much unique and total items have been viewed during a session

In [8]:
item_mapped_sessions = train_sessions.rdd.map(lambda x: (x.session_id, x.item_id)).cache()
uniquely_viewed_items = item_mapped_sessions.groupByKey().mapValues(lambda vals: len(set(vals)))

uniquely_viewed_items.take(10)

[(24, 8),
 (48, 2),
 (184, 1),
 (208, 1),
 (232, 1),
 (248, 10),
 (352, 2),
 (376, 4),
 (384, 10),
 (464, 1)]

In [9]:
total_viewed_items = item_mapped_sessions.groupByKey().mapValues(lambda vals: len(list(vals)))
total_viewed_items.take(10)

[(24, 9),
 (48, 2),
 (184, 1),
 (208, 1),
 (232, 1),
 (248, 15),
 (352, 2),
 (376, 5),
 (384, 10),
 (464, 1)]

# Time period sessions evaluation

We will map for each session its day in the week, if it is on weekend, the month, the season and the year.

In [20]:
# Computing the starting date for each session
starting_date_sessions = train_sessions.rdd.map(lambda x: (x.session_id, x.date)).reduceByKey(lambda x, y: min(x, y))

In [28]:
type(starting_date_sessions.take(1)[0])

str

In [32]:
def parse_datetime(timestamp):
    try:
        return datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S.%f')
    except ValueError:
        return datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
    

def get_season(date_time):
    season = (date_time.month - 1) // 3
    season += (date_time.month == 3)&(date_time.day>=20)
    season += (date_time.month == 6)&(date_time.day>=21)
    season += (date_time.month == 9)&(date_time.day>=23)
    season -= 3*int(((date_time.month == 12)&(date_time.day>=21)))
    return season

def get_day_period(date_time):
    '''Converts the date_time into the day of the week.
    
    0 -> Morning (from 6am to 12am)
    1 -> Afternoon (from 12am to 6pm)
    2 -> Evening (from 6pm to 12pm)
    3 -> Night (from 12pm to 6am)
    '''
    return date_time.hour // 4
    
def map_day_period(timestamp):
    # Converts into datetime
    date_time = parse_datetime(timestamp)
    # Computes the season
    season = get_season(date_time)
    # Computes the 
    return (get_day_period(date_time), date_time.weekday(), date_time.month - 1, season, date_time.year)

date_period_sessions = starting_date_sessions.map(lambda x: (x[0], map_day_period(x[1])))
date_period_sessions.take(10)

[(24, (4, 2, 1, 0, 2020)),
 (48, (4, 2, 3, 1, 2020)),
 (184, (0, 6, 3, 1, 2021)),
 (208, (2, 6, 11, 3, 2020)),
 (232, (3, 4, 8, 2, 2020)),
 (248, (3, 0, 11, 3, 2020)),
 (352, (4, 0, 5, 1, 2020)),
 (376, (2, 5, 0, 0, 2020)),
 (384, (2, 4, 11, 3, 2020)),
 (464, (2, 3, 5, 1, 2020))]